In [8]:
!python -m onmt.bin.translate \
    -model models/model_step_5000.pt \
    -src test.tok.ko \
    -output output.tok.en \
    -beam_size 5 \
    -gpu 0

^C


In [1]:
import sentencepiece as spm

sp = spm.SentencePieceProcessor()
sp.load('sentencepiece.model')

print(sp.encode_as_pieces("안녕하세요"))
print(sp.encode_as_pieces("한국어 문장 테스트입니다."))


['▁안녕', '하세요']
['▁한국어', '▁문', '장', '▁테스트', '입니다', '.']


In [2]:
from model_class import KoreanEnglishTranslator

c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sparse_activations.py:46: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sparse_activations.py:66: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd
c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sru.py:395: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @custom_fwd
c:\Anaconda3\envs\daycon\Lib\site-packages\onmt\modules\sru.py:444: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
  @custom_bwd


In [4]:
global_translator = KoreanEnglishTranslator(
    tokenizer_path='sentencepiece.model',
    model_path='models/model_step_3000.pt'
)

In [6]:
def evaluate_translation(translator, test_file_ko, test_file_en):
    """번역 성능 평가 (BLEU 스코어)"""
    from nltk.translate.bleu_score import corpus_bleu
    
    references = []
    predictions = []
    
    with open(test_file_ko, 'r', encoding='utf-8') as f_ko, \
         open(test_file_en, 'r', encoding='utf-8') as f_en:
        
        for ko_line, en_line in zip(f_ko, f_en):
            korean_text = ko_line.strip()
            reference_english = en_line.strip()
            
            # 번역 실행
            predicted_english = translator.translate(korean_text)
            
            # BLEU 계산용 데이터 준비
            references.append([reference_english.split()])
            predictions.append(predicted_english.split())
    
    # BLEU 스코어 계산
    bleu_score = corpus_bleu(references, predictions)
    print(f"BLEU Score: {bleu_score:.4f}")
    
    return bleu_score

# 평가 실행
bleu = evaluate_translation(global_translator, 'test.ko', 'test.en')
print(bleu)



👉 token_ids: [55, 13526, 28926, 28041, 3335, 18758, 12125, 8144, 227, 29253, 13663, 4178, 9829, 2075, 2662, 1037, 615, 1844, 5436, 28906, 653, 29243, 17580, 2015, 9984, 1219, 28876, 28907, 28962, 28890, 28961, 28928, 5864, 8377, 729, 28895]


AttributeError: 'NMTModel' object has no attribute 'src_emb'

In [5]:
print(dir(global_translator.translator.model))


['T_destination', '__annotations__', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply', '_backward_hooks', '_backward_pre_hooks', '_buffers', '_call_impl', '_compiled_call_impl', '_forward_hooks', '_forward_hooks_always_called', '_forward_hooks_with_kwargs', '_forward_pre_hooks', '_forward_pre_hooks_with_kwargs', '_get_backward_hooks', '_get_backward_pre_hooks', '_get_name', '_is_full_backward_hook', '_load_from_state_dict', '_load_param', '_load_state_dict_post_hooks', '_load_state_dict_pre_hooks', '_maybe_warn_non_full_backward_hook', '_modules', '_named_members', '_non_persistent_buffers_set', '_parameters', '_register_load_sta

In [7]:
vocab_size = None

# 1) encoder 안에 embeddings 속성 있는지 확인
if hasattr(global_translator.translator.model.encoder, 'embeddings'):
    emb = global_translator.translator.model.encoder.embeddings
    # embeddings 내부 구조에 따라 다름
    if hasattr(emb, 'word_lut'):
        vocab_size = emb.word_lut.weight.size(0)
    elif hasattr(emb, 'weight'):
        vocab_size = emb.weight.size(0)



if vocab_size is None:
    raise RuntimeError("vocab size를 찾을 수 없습니다.")

print("모델 vocab size:", vocab_size)

모델 vocab size: 19688
